# ¡Llena ese carrito!

# Introducción

Instacart es una plataforma de entregas de comestibles donde la clientela puede registrar un pedido y hacer que se lo entreguen, similar a Uber Eats y Door Dash.
El conjunto de datos que te hemos proporcionado tiene modificaciones del original. Redujimos el tamaño del conjunto para que tus cálculos se hicieran más rápido e introdujimos valores ausentes y duplicados. Tuvimos cuidado de conservar las distribuciones de los datos originales cuando hicimos los cambios.

Debes completar tres pasos. Para cada uno de ellos, escribe una breve introducción que refleje con claridad cómo pretendes resolver cada paso, y escribe párrafos explicatorios que justifiquen tus decisiones al tiempo que avanzas en tu solución.  También escribe una conclusión que resuma tus hallazgos y elecciones.


## Diccionario de datos

Hay cinco tablas en el conjunto de datos, y tendrás que usarlas todas para hacer el preprocesamiento de datos y el análisis exploratorio de datos. A continuación se muestra un diccionario de datos que enumera las columnas de cada tabla y describe los datos que contienen.

- `instacart_orders.csv`: cada fila corresponde a un pedido en la aplicación Instacart.
    - `'order_id'`: número de ID que identifica de manera única cada pedido.
    - `'user_id'`: número de ID que identifica de manera única la cuenta de cada cliente.
    - `'order_number'`: el número de veces que este cliente ha hecho un pedido.
    - `'order_dow'`: día de la semana en que se hizo el pedido (0 si es domingo).
    - `'order_hour_of_day'`: hora del día en que se hizo el pedido.
    - `'days_since_prior_order'`: número de días transcurridos desde que este cliente hizo su pedido anterior.
- `products.csv`: cada fila corresponde a un producto único que pueden comprar los clientes.
    - `'product_id'`: número ID que identifica de manera única cada producto.
    - `'product_name'`: nombre del producto.
    - `'aisle_id'`: número ID que identifica de manera única cada categoría de pasillo de víveres.
    - `'department_id'`: número ID que identifica de manera única cada departamento de víveres.
- `order_products.csv`: cada fila corresponde a un artículo pedido en un pedido.
    - `'order_id'`: número de ID que identifica de manera única cada pedido.
    - `'product_id'`: número ID que identifica de manera única cada producto.
    - `'add_to_cart_order'`: el orden secuencial en el que se añadió cada artículo en el carrito.
    - `'reordered'`: 0 si el cliente nunca ha pedido este producto antes, 1 si lo ha pedido.
- `aisles.csv`
    - `'aisle_id'`: número ID que identifica de manera única cada categoría de pasillo de víveres.
    - `'aisle'`: nombre del pasillo.
- `departments.csv`
    - `'department_id'`: número ID que identifica de manera única cada departamento de víveres.
    - `'department'`: nombre del departamento.

# Paso 1. Descripción de los datos

Lee los archivos de datos (`/datasets/instacart_orders.csv`, `/datasets/products.csv`, `/datasets/aisles.csv`, `/datasets/departments.csv` y `/datasets/order_products.csv`) con `pd.read_csv()` usando los parámetros adecuados para leer los datos correctamente. Verifica la información para cada DataFrame creado.


## Plan de solución

1. Apertura e inspeccion visual para darse una idea general, si los archivos csv cuentan con que tipo de separador csv, nombres de columnas, formato de nombres de columnas, son consistentes en el numero de columnas y sus titulos, asi como reconocimiento de valores vacios, caracteres raros, valores numericos como strings, con caracteres especiales, etc. 
2. Revision general del dataset, verificar el tamaño, columnas y tipos de datos (int, float, string, datetime, etc.). Comprobar que el numero de filas tenga sentido respecto a lo esperado. 
3. Creacion de DataFrames a partir de la lectura de los archivos CSV.
4. Revision de la informacion de los Dataframes, composicion (tamaño), cantidad de valores nulos, dtypes.
5. Documentacion de cada paso, registro de transformaciones realizadas, esto ayuda a mantener la trazabilidad y reproducibilidad.

In [ ]:
# Importar librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

aisles_path = 'datasets/aisles.csv'
departments_path = 'datasets/departments.csv'
instacart_orders_path = 'datasets/instacart_orders.csv'
order_products_path = 'datasets/order_products.csv'
products_path = 'datasets/products.csv'

# Funcion que da estilo de formato para visualizacion de los DataFrames en Jupyter Notebook 
def zebra_rows(row):
    
    if row.name % 2 == 0:
        return ['background-color: #808080'] * len(row)  # 50% grey
    else:
        return ['background-color: #595959'] * len(row)  # 35% grey

# Funcion para mostrar valores unicos de las columnas de un DataFrame
def columns_unique_values(df):
    
    for col in df.columns:
        print(f"Columna: {col}")
        print(f"Valores únicos: {df[col].unique()} {df[col].nunique()}")
        print("-" * 50)

# Funcion para asignacion de valor <NA> a los valores nulos de un DataFrame
def null_value_apply(df):
    
    null_values = ['', ' ', '-', 'N/A', 'NA', 'NaN', 'nan', 'NULL', 'null', 'None', None]
    
    for col in df.columns:
        
        df[col] = df[col].replace(null_values, pd.NA)
                
    return df

# Funcion para convertir valores 

# Funcion para la estandarizacion de valores String (snake_case, lower, strip, eliminacion de guiones medios)
def string_std(df):
        
    for col in df.columns:
            
        if df[col].dtype == 'object':
                
            df[col] = df[col].str.replace('-', '_').str.lower().str.strip()
                
    return df

pd.set_option('display.max_rows', 20) # Mostrar 10 filas como máximo
pd.set_option('display.max_columns', 10) # Mostrar 10 columnas como máximo
pd.set_option('display.width', 150) # Mostrar 1000 caracteres como máximo

##### __Nota__
Se agregaron en esta seccion las funciones a usar mas adelante, para seguir el formato estandard segun las buenas practicas de programacion.

In [ ]:
# Leer conjuntos de datos en los DataFrames

df_aisles = pd.read_csv(aisles_path, sep=';', header='infer', keep_default_na=False)
df_departments = pd.read_csv(departments_path, sep=';', header='infer', keep_default_na=False)
df_instacart_orders = pd.read_csv(instacart_orders_path, sep=';', header='infer', keep_default_na=False)
df_order_products = pd.read_csv(order_products_path, sep=';', header='infer', keep_default_na=False)
df_products = pd.read_csv(products_path, sep=';', header='infer', keep_default_na=False)

df_aisles.head(20).style.apply(zebra_rows, axis=1)
df_departments.head(20).style.apply(zebra_rows, axis=1)
df_instacart_orders.head(20).style.apply(zebra_rows, axis=1)
df_order_products.head(20).style.apply(zebra_rows, axis=1)
df_products.head(20).style.apply(zebra_rows, axis=1)

##### __Nota__

Se utilizo _header='infer'_, ya que con la inspeccion visual de los archivos csv se identifico que los archivos contaban con el nombre de las columnas en la primera posicion de las filas.
Se utilizo _keep_default_na=False_, ya que posteriormente se piensa identificar,  modificar los valores nulos y reemplazarlo con el valor nulo pd.NA ya que este maneja tipos de datos mixtos, y operaciones comunes para la identificacion y eliminacion de valores nulos, adicionalmente que trabaja con strings, enteros y booleanos.

##### __Infromacion de los DataFrames__

In [ ]:
# Mostrar información del DataFrame

print("*** Aisles DataFrame: \n")
df_aisles.info()
print('*' * 50)
print("*** Departments DataFrame:\n")
df_departments.info()
print('*' * 50)
print("*** Instacart Orders DataFrame:\n")
df_instacart_orders.info()
print('*' * 50)
print("*** Order Products DataFrame:\n")
df_order_products.info()
print('*' * 50)
print("*** Products DataFrame:\n")
df_products.info()
print('*' * 50)


##### __Nota__

No sera necesario realizar ningun tipo de porcesamiento para el nombre de las columnas ya que estan estandarizadas, con minusculas, usando guion bajo para la separacion de los nombres compuestos, no cuentan con espacios ni al inicio ni al final de cada nombre de columna. 

##### __Valores Nulos__

In [ ]:
# Mostrar información del DataFrame para valores nulos

print("*** DataFrame Aisles: \n")
columns_unique_values(df_aisles)
print()
print("*** DataFrame Departments: \n")
columns_unique_values(df_departments)
print()
print("*** DataFrame Instacart Orders: \n")
columns_unique_values(df_instacart_orders)
print()
print("*** DataFrame Order Products: \n")
columns_unique_values(df_order_products)
print()
print("*** DataFrame Products: \n")
columns_unique_values(df_products)


In [ ]:
# Identificacion y correcion de valores Null a pd.NA

null_value_apply( df_aisles)
null_value_apply( df_departments)
null_value_apply( df_instacart_orders)
null_value_apply( df_order_products)
null_value_apply( df_products)

print("Data Frame aisle valores nulos: \n", df_aisles.isna().sum())
print('*' * 50)
print("Data Frame department valores nulos: \n", df_departments.isna().sum())
print('*' * 50)
print("Data Frame instacart_orders valores nulos: \n", df_instacart_orders.isna().sum())
print('*' * 50)
print("Data Frame order_products valores nulos: \n", df_order_products.isna().sum())
print('*' * 50)
print("Data Frame products valores nulos: \n", df_products.isna().sum())
print('*' * 50)
print('*' * 50)
print()
print("*** Valores Nulos debidamene identificados y corregidos como <NA>: \n")
print("*** DataFrame Instacart Orders: \n")
columns_unique_values(df_instacart_orders)
print()
print("*** DataFrame Order Products: \n")
columns_unique_values(df_order_products)
print()
print("*** DataFrame Products: \n")
columns_unique_values(df_products)


##### __Estandarizacion de valores String__

In [ ]:
# mostrar información del DataFrame

## Conclusiones

El formato y valores del nombre de las columnas no requieren de modificacion.
Para la limpieza o preprocesamiento de valores de los DataFrames requeriran:

- Estandarizacion de formato a snake_case considerando espacion y guiones medios entre strings compuestos.
- Procesamiento de valores NULL
- Eliminacion de valores duplicados
- Procesamiento de valores string a numericos

La mayor parte de los valores de los Dataframe se puede considerar moderadamente correcta, sin mucha dificultad para preprocesarlos.


# Paso 2. Preprocesamiento de los datos

Preprocesa los datos de la siguiente manera:

- Verifica y corrige los tipos de datos (por ejemplo, asegúrate de que las columnas de ID sean números enteros).
- Identifica y completa los valores ausentes.
- Identifica y elimina los valores duplicados.

Asegúrate de explicar qué tipos de valores ausentes y duplicados encontraste, cómo los completaste o eliminaste y por qué usaste esos métodos. ¿Por qué crees que estos valores ausentes y duplicados pueden haber estado presentes en el conjunto de datos?

## Plan de solución

1. Manejo de valores faltantes: Identificar valores nulos (NaN, None, celdas vacías).

2. Analisis y decision para: Eliminacion filas o columnas con muchos valores faltantes. Reemplazo (con media, mediana, 0's, etc.). Mantener si tienen algún significado.

3. Deteccion de duplicados: Eliminacion filas duplicadas con duplicado explicitos. Revision y correccion para duplicados implicitos (errores tipograficos).

4. Consistencia en los formatos: Asegurar valores de fechas esten en el mismo formato. Estandarizacion de categorias que esten escritas diferente (ej. "Femenino", "femenina", "Mujer" → "F"). Conversion datos a los tipos correctos (str, datetime, int, etc.).

5. Verificacion de codificación y caracteres especiales: Revison de existencia de caracteres raros, espacios ocultos.

6. Documentacion de cada paso: Registro de transformaciones realizadas. Esto ayuda a mantener la trazabilidad y reproducibilidad.

## Encuentra y elimina los valores duplicados (y describe cómo tomaste tus decisiones).

### `instacart orders` data frame

In [ ]:
# Revisa si hay pedidos duplicados
print("*** Instacart Orders DataFrame valores duplicados explicitos:\n")
print(f"Duplicados: {df_instacart_orders.duplicated().sum()}")
print('-' * 50)


¿Tienes líneas duplicadas? Si sí, ¿qué tienen en común?

In [ ]:
# Basándote en tus hallazgos,
# Verifica todos los pedidos que se hicieron el miércoles a las 2:00 a.m.

print(df_instacart_orders.loc[(df_instacart_orders['order_hour_of_day'] == 2)  & (df_instacart_orders['order_dow'] == 3), :] )

¿Qué sugiere este resultado?

In [ ]:
# Elimina los pedidos duplicados

In [ ]:
# Vuelve a verificar si hay filas duplicadas

In [ ]:
# Vuelve a verificar únicamente si hay IDs duplicados de pedidos

In [ ]:
# Mostrar valores duplicados explicitos en los DataFrames
print("*** Aisles DataFrame: \n")
print(f"Duplicados: {df_aisles.duplicated().sum()}")
print('-' * 50)
print("*** Departments DataFrame:\n")
print(f"Duplicados: {df_departments.duplicated().sum()}")
print('-' * 50)
print("*** Instacart Orders DataFrame:\n")
print(f"Duplicados: {df_instacart_orders.duplicated().sum()}")
print('-' * 50)
print("*** Order Products DataFrame:\n")
print(f"Duplicados: {df_order_products.duplicated().sum()}")
print('-' * 50)
print("*** Products DataFrame:\n")
print(f"Duplicados: {df_products.duplicated().sum()}")
print('-' * 50)

##### __Nota__


In [ ]:
# Eliminacion de duplicados explicitos en DF instacart_orders
df_instacart_orders = df_instacart_orders.drop_duplicates().reset_index(drop=True)

# Validacion de duplicados explicitos

print("*** Aisles DataFrame: \n")
print(f"Duplicados: {df_aisles.duplicated().sum()}")
print('-' * 50)
print("*** Departments DataFrame:\n")
print(f"Duplicados: {df_departments.duplicated().sum()}")
print('-' * 50)
print("*** Instacart Orders DataFrame:\n")
print(f"Duplicados: {df_instacart_orders.duplicated().sum()}")
print('-' * 50)
print("*** Order Products DataFrame:\n")
print(f"Duplicados: {df_order_products.duplicated().sum()}")
print('-' * 50)
print("*** Products DataFrame:\n")
print(f"Duplicados: {df_products.duplicated().sum()}")
print('-' * 50)

Describe brevemente tus hallazgos y lo que hiciste con ellos

### `products` data frame

In [ ]:
# Verifica si hay filas totalmente duplicadas


In [ ]:
# Revisa únicamente si hay ID de departamentos duplicados


In [ ]:
# Revisa únicamente si hay nombres duplicados de productos (convierte los nombres a letras mayúsculas para compararlos mejor)


In [ ]:
# Revisa si hay nombres duplicados de productos no faltantes


Describe brevemente tus hallazgos y lo que hiciste con ellos.

### `departments` data frame

In [ ]:
# Revisa si hay filas totalmente duplicadas

In [ ]:
# Revisa únicamente si hay IDs duplicadas de productos

Describe brevemente tus hallazgos y lo que hiciste con ellos.

### `aisles` data frame

In [ ]:
# Revisa si hay filas totalmente duplicadas

In [ ]:
# Revisa únicamente si hay IDs duplicadas de pasillos

Describe brevemente tus hallazgos y lo que hiciste con ellos.

### `order_products` data frame

In [ ]:
# Revisa si hay filas totalmente duplicadas


In [ ]:
# Vuelve a verificar si hay cualquier otro duplicado engañoso


Describe brevemente tus hallazgos y lo que hiciste con ellos.

## Encuentra y elimina los valores ausentes

Al trabajar con valores duplicados, pudimos observar que también nos falta investigar valores ausentes:

* La columna `'product_name'` de la tabla products.
* La columna `'days_since_prior_order'` de la tabla orders.
* La columna `'add_to_cart_order'` de la tabla order_productos.

### `products` data frame

In [ ]:
# Encuentra los valores ausentes en la columna 'product_name'

Describe brevemente cuáles son tus hallazgos.

In [ ]:
#  ¿Todos los nombres de productos ausentes están relacionados con el pasillo con ID 100?


Describe brevemente cuáles son tus hallazgos.

In [ ]:
# ¿Todos los nombres de productos ausentes están relacionados con el departamento con ID 21?


Describe brevemente cuáles son tus hallazgos.

In [ ]:
# Usa las tablas department y aisle para revisar los datos del pasillo con ID 100 y el departamento con ID 21.


Describe brevemente cuáles son tus hallazgos.

In [ ]:
# Completa los nombres de productos ausentes con 'Unknown'


Describe brevemente tus hallazgos y lo que hiciste con ellos.

### `orders` data frame

In [ ]:
# Encuentra los valores ausentes

In [ ]:
# ¿Hay algún valor ausente que no sea el primer pedido del cliente?


Describe brevemente tus hallazgos y lo que hiciste con ellos.

### `order_products` data frame

In [ ]:
# Encuentra los valores ausentes

In [ ]:
# ¿Cuáles son los valores mínimos y máximos en esta columna?


Describe brevemente cuáles son tus hallazgos.

In [ ]:
# Guarda todas las IDs de pedidos que tengan un valor ausente en 'add_to_cart_order'


In [ ]:
# ¿Todos los pedidos con valores ausentes tienen más de 64 productos?
# Agrupa todos los pedidos con datos ausentes por su ID de pedido.
# Cuenta el número de 'product_id' en cada pedido y revisa el valor mínimo del conteo.


Describe brevemente cuáles son tus hallazgos.

In [ ]:
# Remplaza los valores ausentes en la columna 'add_to_cart? con 999 y convierte la columna al tipo entero.


Describe brevemente tus hallazgos y lo que hiciste con ellos.

## Conclusiones

Escribe aquí tus conclusiones intermedias sobre el Paso 2. Preprocesamiento de los datos


# Paso 3. Análisis de los datos

Una vez los datos estén procesados y listos, haz el siguiente análisis:

# [A] Fácil (deben completarse todos para aprobar)

1. Verifica que los valores en las columnas `'order_hour_of_day'` y `'order_dow'` en la tabla orders sean razonables (es decir, `'order_hour_of_day'` oscile entre 0 y 23 y `'order_dow'` oscile entre 0 y 6).
2. Crea un gráfico que muestre el número de personas que hacen pedidos dependiendo de la hora del día.
3. Crea un gráfico que muestre qué día de la semana la gente hace sus compras.
4. Crea un gráfico que muestre el tiempo que la gente espera hasta hacer su siguiente pedido, y comenta sobre los valores mínimos y máximos.

### [A1] Verifica que los valores sean sensibles

Escribe aquí tus conclusiones

### [A2] Para cada hora del día, ¿cuántas personas hacen órdenes?

Escribe aquí tus conclusiones

### [A3] ¿Qué día de la semana compran víveres las personas?

Escribe aquí tus conclusiones

### [A4] ¿Cuánto tiempo esperan las personas hasta hacer otro pedido? Comenta sobre los valores mínimos y máximos.

Escribe aquí tus conclusiones

# [B] Intermedio (deben completarse todos para aprobar)

1. ¿Existe alguna diferencia entre las distribuciones `'order_hour_of_day'` de los miércoles y los sábados? Traza gráficos de barra de `'order_hour_of_day'` para ambos días en la misma figura y describe las diferencias que observes.
2. Grafica la distribución para el número de órdenes que hacen los clientes (es decir, cuántos clientes hicieron solo 1 pedido, cuántos hicieron 2, cuántos 3, y así sucesivamente...).
3. ¿Cuáles son los 20 principales productos que se piden con más frecuencia (muestra su identificación y nombre)?

### [B1] Diferencia entre miércoles y sábados para  `'order_hour_of_day'`. Traza gráficos de barra para los dos días y describe las diferencias que veas.

Escribe aquí tus conclusiones

### [B2] ¿Cuál es la distribución para el número de pedidos por cliente?

Escribe aquí tus conclusiones

### [B3] ¿Cuáles son los 20 productos más populares (muestra su ID y nombre)?

Escribe aquí tus conclusiones

# [C] Difícil (deben completarse todos para aprobar)

1. ¿Cuántos artículos suelen comprar las personas en un pedido? ¿Cómo es la distribución?
2. ¿Cuáles son los 20 principales artículos que vuelven a pedirse con mayor frecuencia (muestra sus nombres e IDs de los productos)?
3. Para cada producto, ¿cuál es la tasa de repetición del pedido (número de repeticiones de pedido/total de pedidos?
4. Para cada cliente, ¿qué proporción de los productos que pidió ya los había pedido? Calcula la tasa de repetición de pedido para cada usuario en lugar de para cada producto.
5. ¿Cuáles son los 20 principales artículos que la gente pone primero en sus carritos (muestra las IDs de los productos, sus nombres, y el número de veces en que fueron el primer artículo en añadirse al carrito)?

### [C1] ¿Cuántos artículos compran normalmente las personas en un pedido? ¿Cómo es la distribución?

Escribe aquí tus conclusiones

### [C2] ¿Cuáles son los 20 principales artículos que vuelven a pedirse con mayor frecuencia (muestra sus nombres e IDs de los productos)?

Escribe aquí tus conclusiones

### [C3] Para cada producto, ¿cuál es la proporción de las veces que se pide y que se vuelve a pedir?

Escribe aquí tus conclusiones

### [C4] Para cada cliente, ¿qué proporción de sus productos ya los había pedido?

Escribe aquí tus conclusiones

### [C5] ¿Cuáles son los 20 principales artículos que las personas ponen primero en sus carritos?

Escribe aquí tus conclusiones

### Conclusion general del proyecto: